In [8]:
import os
import numpy as np
import tensorflow as tf
import h5py
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.metrics import MeanIoU
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define image parameters
img_width, img_height = 384, 384  # Matches your dataset dimensions
input_shape = (img_width, img_height, 1)  # Grayscale images
batch_size = 8
epochs = 50

# Load CAMUS dataset from HDF5 file
def load_camus_data(hdf5_path):
    with h5py.File(hdf5_path, 'r') as f:
        # Load 2CH and 4CH views
        train_2ch_frames = np.array(f['train 2ch frames'])
        train_2ch_masks = np.array(f['train 2ch masks'])
        train_4ch_frames = np.array(f['train 4ch frames'])
        train_4ch_masks = np.array(f['train 4ch masks'])
        
        # Combine all frames and masks
        all_frames = np.concatenate([train_2ch_frames, train_4ch_frames], axis=0)
        all_masks = np.concatenate([train_2ch_masks, train_4ch_masks], axis=0)
        
        # Shuffle the data
        indices = np.arange(len(all_frames))
        np.random.shuffle(indices)
        all_frames = all_frames[indices]
        all_masks = all_masks[indices]
        
        # Split into train/val/test (70%/15%/15%)
        train_size = int(0.7 * len(all_frames))
        val_size = int(0.15 * len(all_frames))
        
        train_images = all_frames[:train_size]
        train_masks = all_masks[:train_size]
        
        val_images = all_frames[train_size:train_size+val_size]
        val_masks = all_masks[train_size:train_size+val_size]
        
        test_images = all_frames[train_size+val_size:]
        test_masks = all_masks[train_size+val_size:]
        
    return (train_images, train_masks), (val_images, val_masks), (test_images, test_masks)

# Load the data
try:
    hdf5_path = "../input/camus-dataset/image_dataset.hdf5"
    print(f"Loading data from: {hdf5_path}")
    (train_images, train_masks), (val_images, val_masks), (test_images, test_masks) = load_camus_data(hdf5_path)
    
    print("\nData loaded successfully!")
    print(f"Training images shape: {train_images.shape}")
    print(f"Training masks shape: {train_masks.shape}")
    print(f"Validation images shape: {val_images.shape}")
    print(f"Test images shape: {test_images.shape}")
    
except Exception as e:
    print(f"\nError loading data: {e}")
    raise

# Data augmentation function
def augment_data(images, masks):
    # Convert masks to float32 for augmentation
    masks = masks.astype('float32')
    
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest')
    
    seed = 42
    image_generator = datagen.flow(images, seed=seed, batch_size=batch_size)
    mask_generator = datagen.flow(masks, seed=seed, batch_size=batch_size)
    
    while True:
        yield (next(image_generator), next(mask_generator))

# Normalize images to [0,1] and masks to binary (0 or 1)
train_images = train_images.astype('float32') / 255.0
train_masks = (train_masks > 0).astype('float32')  # Convert to binary masks

val_images = val_images.astype('float32') / 255.0
val_masks = (val_masks > 0).astype('float32')

test_images = test_images.astype('float32') / 255.0
test_masks = (test_masks > 0).astype('float32')

# Create data generators
train_generator = augment_data(train_images, train_masks)

# Simple generator for validation (no augmentation)
def val_data_generator(images, masks):
    while True:
        for i in range(0, len(images), batch_size):
            yield (images[i:i+batch_size], masks[i:i+batch_size])

val_generator = val_data_generator(val_images, val_masks)

# Build U-Net model
def build_unet(input_shape):
    inputs = Input(input_shape)
    
    # Downsample path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Bottleneck
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    
    # Upsample path
    up4 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv3)
    up4 = concatenate([up4, conv2])
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(up4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)
    
    up5 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv4)
    up5 = concatenate([up5, conv1])
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(up5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)
    
    outputs = Conv2D(1, 1, activation='sigmoid')(conv5)
    
    return Model(inputs=[inputs], outputs=[outputs])

model = build_unet(input_shape)
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy', MeanIoU(num_classes=2)])

model.summary()

# Callbacks
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ModelCheckpoint('camus_best_model.h5', verbose=1, save_best_only=True)
]

# Calculate steps per epoch
steps_per_epoch = len(train_images) // batch_size
validation_steps = len(val_images) // batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=callbacks)

# Evaluation on test set
test_results = model.evaluate(test_images, test_masks, batch_size=batch_size)
print(f"Test Loss: {test_results[0]}, Test Accuracy: {test_results[1]}, Test IoU: {test_results[2]}")

# Save the final model
model.save('camus_echo_segmentation_final.h5')

print("Training completed and model saved!")

Loading data from: ../input/camus-dataset/image_dataset.hdf5

Data loaded successfully!
Training images shape: (1260, 384, 384, 1)
Training masks shape: (1260, 384, 384, 1)
Validation images shape: (270, 384, 384, 1)
Test images shape: (270, 384, 384, 1)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384, 384, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 384, 384, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                     

**Note:** In this notebook, I did replication of https://github.com/albergcg/camus_challenge and I tried something different that I was curious about. I trained model step by step without data leakage.

If you encounter with any error, you can find the dataset from following links:

ready-to-use version of camus: https://www.kaggle.com/datasets/toygarr/camus-dataset<br/>
subject-based splitted version of camus: https://www.kaggle.com/datasets/toygarr/camus-subject-based<br/>
original dataset: https://humanheart-project.creatis.insa-lyon.fr/database/#collection/6373703d73e9f0047faa1bc8

In [2]:
import os
import numpy as np
import tensorflow as tf
import h5py
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
                                   Conv2DTranspose, concatenate, Input, LSTM, Reshape)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.metrics import MeanIoU
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# 1. Data Loading and Preparation
def load_camus_data(hdf5_path):
    with h5py.File(hdf5_path, 'r') as f:
        # Load 2CH and 4CH views
        train_2ch_frames = np.array(f['train 2ch frames'])
        train_2ch_masks = np.array(f['train 2ch masks'])
        train_4ch_frames = np.array(f['train 4ch frames'])
        train_4ch_masks = np.array(f['train 4ch masks'])
        
        # Create labels (0 for 2CH, 1 for 4CH)
        train_2ch_labels = np.zeros(len(train_2ch_frames))
        train_4ch_labels = np.ones(len(train_4ch_frames))
        
        # Combine all frames, masks and labels
        all_frames = np.concatenate([train_2ch_frames, train_4ch_frames], axis=0)
        all_masks = np.concatenate([train_2ch_masks, train_4ch_masks], axis=0)
        all_labels = np.concatenate([train_2ch_labels, train_4ch_labels], axis=0)
        
        # Split into train (70%), val (15%), test (15%)
        X_train, X_temp, y_train, y_temp, label_train, label_temp = train_test_split(
            all_frames, all_masks, all_labels, test_size=0.3, random_state=42)
        X_val, X_test, y_val, y_test, label_val, label_test = train_test_split(
            X_temp, y_temp, label_temp, test_size=0.5, random_state=42)
        
    return (X_train, y_train, label_train), (X_val, y_val, label_val), (X_test, y_test, label_test)

# Load the data
hdf5_path = "../input/camus-dataset/image_dataset.hdf5"
(X_train, y_train, label_train), (X_val, y_val, label_val), (X_test, y_test, label_test) = load_camus_data(hdf5_path)

# Normalize images and convert masks to binary
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

y_train = (y_train > 0).astype('float32')
y_val = (y_val > 0).astype('float32')
y_test = (y_test > 0).astype('float32')

# Convert labels to categorical
label_train = to_categorical(label_train, num_classes=2)
label_val = to_categorical(label_val, num_classes=2)
label_test = to_categorical(label_test, num_classes=2)

# 2. Data Generators
# Corrected Data Generator Implementation
def create_generator(images, masks, labels=None, batch_size=8, augment=False):
    if augment:
        data_gen_args = dict(
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            zoom_range=0.1,
            horizontal_flip=True,
            fill_mode='nearest')
    else:
        data_gen_args = dict()
    
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    
    seed = 42
    image_generator = image_datagen.flow(images, seed=seed, batch_size=batch_size)
    mask_generator = mask_datagen.flow(masks, seed=seed, batch_size=batch_size)
    
    if labels is not None:
        # Reshape labels to 4D (batch_size, 1, 1, num_classes) for ImageDataGenerator
        labels_reshaped = labels.reshape(-1, 1, 1, labels.shape[1])
        label_datagen = ImageDataGenerator(**data_gen_args)
        label_generator = label_datagen.flow(labels_reshaped, seed=seed, batch_size=batch_size)
        
        while True:
            # Get the next batch from each generator
            x_batch = next(image_generator)
            y_mask = next(mask_generator)
            y_class = next(label_generator)
            
            # Squeeze the class labels back to 2D
            y_class = y_class.reshape(-1, labels.shape[1])
            
            yield (x_batch, {'seg_output': y_mask, 'class_output': y_class})
    else:
        while True:
            yield (next(image_generator), next(mask_generator))

# Update the generators with the corrected implementation
batch_size = 8
train_generator = create_generator(X_train, y_train, label_train, batch_size, augment=True)
val_generator = create_generator(X_val, y_val, label_val, batch_size)
      

# 3. U-Net Model for Segmentation
def build_unet(input_shape):
    inputs = Input(input_shape)
    
    # Downsample path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Bottleneck
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    
    # Upsample path
    up4 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv3)
    up4 = concatenate([up4, conv2])
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(up4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)
    
    up5 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv4)
    up5 = concatenate([up5, conv1])
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(up5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)
    
    seg_output = Conv2D(1, 1, activation='sigmoid', name='seg_output')(conv5)
    
    # Classification branch
    gap = tf.keras.layers.GlobalAveragePooling2D()(conv5)
    class_output = Dense(2, activation='softmax', name='class_output')(gap)
    
    return Model(inputs=[inputs], outputs=[seg_output, class_output])

# Build and compile multi-output model
model = build_unet((384, 384, 1))
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss={'seg_output': 'binary_crossentropy', 'class_output': 'categorical_crossentropy'},
              metrics={'seg_output': ['accuracy', MeanIoU(num_classes=2)],
                       'class_output': ['accuracy']})

# Callbacks
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ModelCheckpoint('camus_best_model.h5', verbose=1, save_best_only=True)
]

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // batch_size,
    epochs=50,
    validation_data=val_generator,
    validation_steps=len(X_val) // batch_size,
    callbacks=callbacks)

# Save models
model.save('camus_segmentation_classification_model.h5')
print("Segmentation and classification model saved")

# 4. RNN Model for Clinical Decision
def prepare_rnn_input(seg_results, class_results):
    # Extract features from segmentation results
    seg_features = np.array([np.mean(seg_results, axis=(1, 2, 3))]).T  # Mean activation
    
    # Combine with classification probabilities
    features = np.concatenate([seg_features, class_results], axis=1)
    
    # Reshape for RNN (samples, timesteps, features)
    return np.reshape(features, (features.shape[0], 1, features.shape[1]))

# Generate synthetic labels for demonstration (0 = normal, 1 = abnormal)
# In practice, you should use real clinical labels
rnn_labels = np.random.randint(0, 2, size=(len(X_train),))
rnn_labels = to_categorical(rnn_labels, num_classes=2)

# Prepare RNN training data
seg_pred, class_pred = model.predict(X_train, batch_size=batch_size)
rnn_input = prepare_rnn_input(seg_pred, class_pred)

def create_rnn_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy',
                 optimizer=Adam(learning_rate=1e-3),
                 metrics=['accuracy'])
    return model

rnn_model = create_rnn_model((1, 3))  # 1 timestep, 3 features (seg feature + 2 class probs)

# Train RNN
rnn_history = rnn_model.fit(
    rnn_input, 
    rnn_labels,
    batch_size=32,
    epochs=20,
    validation_split=0.2)

# Save RNN model
rnn_model.save('camus_rnn_model.h5')
print("RNN model saved")

# 5. Clinical Report Generation
def generate_clinical_report(image):
    # Expand dimensions if single image
    if len(image.shape) == 3:
        image = np.expand_dims(image, axis=0)
    
    # Get model predictions
    seg_pred, class_pred = model.predict(image)
    
    # Prepare RNN input
    rnn_input = prepare_rnn_input(seg_pred, class_pred)
    
    # Get clinical decision
    rnn_pred = rnn_model.predict(rnn_input)
    decision = np.argmax(rnn_pred, axis=1)[0]
    
    # Generate report
    chamber_type = "2-chamber" if np.argmax(class_pred[0]) == 0 else "4-chamber"
    severity = np.mean(seg_pred)
    
    report = f"Echocardiogram Analysis Report:\n"
    report += f"View: {chamber_type}\n"
    report += f"Segmentation coverage: {severity:.2f}\n"
    
    if decision == 0:
        report += "Conclusion: NORMAL - No significant abnormalities detected."
    else:
        if severity < 0.3:
            report += "Conclusion: MILD ABNORMALITY - Recommend follow-up."
        elif severity < 0.6:
            report += "Conclusion: MODERATE ABNORMALITY - Recommend cardiology consultation."
        else:
            report += "Conclusion: SEVERE ABNORMALITY - Urgent intervention required."
    
    return report

# Example usage
sample_report = generate_clinical_report(X_test[0])
print("\n" + "="*50)
print(sample_report)
print("="*50)

# 6. Evaluation
def evaluate_models():
    # Evaluate segmentation and classification
    seg_pred, class_pred = model.predict(X_test, batch_size=batch_size)
    seg_accuracy = np.mean((seg_pred > 0.5) == y_test)
    class_accuracy = np.mean(np.argmax(class_pred, axis=1) == np.argmax(label_test, axis=1))
    
    print(f"\nSegmentation Accuracy: {seg_accuracy:.4f}")
    print(f"Classification Accuracy: {class_accuracy:.4f}")
    
    # Evaluate RNN (with synthetic labels)
    rnn_test_input = prepare_rnn_input(seg_pred, class_pred)
    rnn_test_labels = to_categorical(np.random.randint(0, 2, size=(len(X_test),)), num_classes=2)
    rnn_loss, rnn_acc = rnn_model.evaluate(rnn_test_input, rnn_test_labels)
    print(f"RNN Accuracy: {rnn_acc:.4f}")

evaluate_models()

/opt/conda/lib/python3.10/site-packages/keras/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (1260, 1, 1, 2) (2 channels).
  warnings.warn(


Epoch 1/50
157/157 [==============================] - ETA: 0s - loss: 1.2748 - seg_output_loss: 0.5789 - class_output_loss: 0.6959 - seg_output_accuracy: 0.7536 - seg_output_mean_io_u_1: 0.3801 - class_output_accuracy: 0.4873

/opt/conda/lib/python3.10/site-packages/keras/preprocessing/image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (270, 1, 1, 2) (2 channels).
  warnings.warn(



Epoch 1: val_loss improved from inf to 1.21859, saving model to camus_best_model.h5
157/157 [==============================] - 129s 678ms/step - loss: 1.2748 - seg_output_loss: 0.5789 - class_output_loss: 0.6959 - seg_output_accuracy: 0.7536 - seg_output_mean_io_u_1: 0.3801 - class_output_accuracy: 0.4873 - val_loss: 1.2186 - val_seg_output_loss: 0.5253 - val_class_output_loss: 0.6933 - val_seg_output_accuracy: 0.7558 - val_seg_output_mean_io_u_1: 0.3779 - val_class_output_accuracy: 0.4773
Epoch 2/50
157/157 [==============================] - ETA: 0s - loss: 1.1132 - seg_output_loss: 0.4153 - class_output_loss: 0.6980 - seg_output_accuracy: 0.7537 - seg_output_mean_io_u_1: 0.3801 - class_output_accuracy: 0.4936
Epoch 2: val_loss improved from 1.21859 to 1.14692, saving model to camus_best_model.h5
157/157 [==============================] - 114s 679ms/step - loss: 1.1132 - seg_output_loss: 0.4153 - class_output_loss: 0.6980 - seg_output_accuracy: 0.7537 - seg_output_mean_io_u_1: 0.3801